In [17]:
import pandas as pd
import numpy as np
path='C:\\Users\\zhangchen20\\Desktop\\time_test.xlsx'
file_practice=pd.read_excel(path)

In [7]:
file_practice.keys()

Index(['app_ord_dp_overseas_coupon_quantity.product_id',
       'app_ord_dp_overseas_coupon_quantity.platform',
       'app_ord_dp_overseas_coupon_quantity.city_id',
       'app_ord_dp_overseas_coupon_quantity.city_name',
       'app_ord_dp_overseas_coupon_quantity.couponsource',
       'app_ord_dp_overseas_coupon_quantity.coupontitle',
       'app_ord_dp_overseas_coupon_quantity.region',
       'app_ord_dp_overseas_coupon_quantity.getuser',
       'app_ord_dp_overseas_coupon_quantity.useuser',
       'app_ord_dp_overseas_coupon_quantity.getnum',
       'app_ord_dp_overseas_coupon_quantity.usenum',
       'app_ord_dp_overseas_coupon_quantity.partition_date'],
      dtype='object')

In [19]:
#pd.to_datetime(file_practice['app_ord_dp_overseas_coupon_quantity.partition_date'])
def date_split(date):
    y,m,d=date.split('-')
    return '%s年-%s月'%(y,m)
Year_Mon=file_practice['app_ord_dp_overseas_coupon_quantity.partition_date'].map(date_split)
file_practice['Year-Mon']=Year_Mon
#file_practice.head(2)

In [20]:
#先设置datetimeindex， 再重采样
file_practice['date']=pd.to_datetime(file_practice['app_ord_dp_overseas_coupon_quantity.partition_date'])

#file_practice['date'].resample('y',how='sum') 这个可以跑一下，看下为什么会有
file2=file_practice
file2=file2.set_index(file2.date).resample('w').sum()


In [21]:
# 根据ID来分组，然后对每组进行重采样

'''
用下面这个是没有用的
def groupbyID_resemple_byweek(group):
    group.resample('w').count()
'''
file_practice.set_index(file_practice.date,inplace=True) #经过测试，折一行是必须的，只有有了DatetimeIndex才可以用dataframe.resample
file_groupbyID_resemple_bywee=file_practice.groupby('app_ord_dp_overseas_coupon_quantity.product_id').resample('w').sum()

In [28]:
#用这个方式，加上 “星期几之后”，再split就可以划分 ‘周末’‘工作日’了 
date_ser = pd.Series([dt.strftime('%y-%m-%d-%a') for dt in file_practice['date']])
date_ser.head(3)

0    18-08-24-Fri
1    18-08-24-Fri
2    18-08-24-Fri
dtype: object

In [35]:
#选取“某时间段内” 的数据， 通过掩码
#如，选取8月之前的数据
auge=datetime.datetime(2018,8,31)
fiel_before_aug=file_practice[file_practice.date<auge]
fiel_before_aug.head(2)

,app_ord_dp_overseas_coupon_quantity.product_id,app_ord_dp_overseas_coupon_quantity.platform,app_ord_dp_overseas_coupon_quantity.city_id,app_ord_dp_overseas_coupon_quantity.city_name,app_ord_dp_overseas_coupon_quantity.couponsource,app_ord_dp_overseas_coupon_quantity.coupontitle,app_ord_dp_overseas_coupon_quantity.region,app_ord_dp_overseas_coupon_quantity.getuser,app_ord_dp_overseas_coupon_quantity.useuser,app_ord_dp_overseas_coupon_quantity.getnum,app_ord_dp_overseas_coupon_quantity.usenum,app_ord_dp_overseas_coupon_quantity.partition_date,Year-Mon,date
date,,,,,,,,,,,,,,
2018-08-24,60021296814,点评,"2401,15162,4262,15134,2395,15168,10079,2397,15...","马萨诸塞州,檀香山,密歇根州,美国,华盛顿,纽约,德克萨斯州,纳帕,加利福尼亚州,拉斯维加斯...",图文优惠券,Tommy Hilfiger(立享85折),欧美,40,0,40,0,2018-08-24,2018年-08月,2018-08-24
2018-08-24,60021296814,美团,"2401,15162,4262,15134,2395,15168,10079,2397,15...","马萨诸塞州,檀香山,密歇根州,美国,华盛顿,纽约,德克萨斯州,纳帕,加利福尼亚州,拉斯维加斯...",图文优惠券,Tommy Hilfiger(立享85折),欧美,5,0,5,0,2018-08-24,2018年-08月,2018-08-24


In [5]:
# 这个只能对单个datetime对象进行处理
import datetime
date=datetime.datetime(2018,4,7)
date.strftime('%Y-%m-%d-%a')

'2018-04-07-Sat'

date
2018-08-24     True
2018-08-24     True
2018-08-24     True
2018-08-24     True
2018-08-24     True
2018-08-25     True
2018-08-25     True
2018-08-25     True
2018-08-25     True
2018-08-25     True
2018-08-25     True
2018-08-26     True
2018-08-26     True
2018-08-26     True
2018-08-26     True
2018-08-26     True
2018-08-26     True
2018-08-27     True
2018-08-27     True
2018-08-27     True
2018-08-27     True
2018-08-27     True
2018-08-27     True
2018-08-28     True
2018-08-28     True
2018-08-28     True
2018-08-28     True
2018-08-28     True
2018-08-28     True
2018-08-29     True
              ...  
2018-11-03    False
2018-11-03    False
2018-11-03    False
2018-11-03    False
2018-11-03    False
2018-11-03    False
2018-11-03    False
2018-11-04    False
2018-11-04    False
2018-11-04    False
2018-11-04    False
2018-11-04    False
2018-11-04    False
2018-11-04    False
2018-11-04    False
2018-11-04    False
2018-11-04    False
2018-11-05    False
2018-11-05    F